In [34]:
import json

import pandas as pd
import numpy as np


In [35]:
df = pd.read_csv("generations_example.csv")
df.head()

,prompt,completion,reward,reference_response_1,reference_response_2,reference_reward,reference_reward_1,reference_reward_2,reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_43_BT_RM_Qwen3-Embedding-8B_916704_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-482,reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_42_BT_RM_Qwen3-Embedding-8B_915487_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-272
0,<|im_start|>user\nExplain following rating of ...,Here is a detailed explanation of each rating ...,-26.648438,Here are the explanations for each of the 9 ra...,The ratings of **SCR (Single-Phase SCR)** are ...,2.503906,2.474609,2.53125,-1.648438,-1.562500
1,<|im_start|>user\nExplain following rating of ...,Here's a detailed explanation of each rating (...,-25.486328,Here are the explanations for each of the 9 ra...,The ratings of **SCR (Single-Phase SCR)** are ...,2.503906,2.474609,2.53125,-0.486328,1.085938
2,<|im_start|>user\nExplain following rating of ...,Here's a summary of the four main parameters i...,-27.437500,Here are the explanations for each of the 9 ra...,The ratings of **SCR (Single-Phase SCR)** are ...,2.503906,2.474609,2.53125,-2.437500,-2.203125
3,<|im_start|>user\nExplain following rating of ...,Here are explanations for each of the 10 stand...,-26.945312,Here are the explanations for each of the 9 ra...,The ratings of **SCR (Single-Phase SCR)** are ...,2.503906,2.474609,2.53125,-1.945312,-0.593750
4,<|im_start|>user\nExplain following rating of ...,Here is a detailed explanation of the rating p...,-24.468750,Here are the explanations for each of the 9 ra...,The ratings of **SCR (Single-Phase SCR)** are ...,2.503906,2.474609,2.53125,0.531250,-1.125000


In [36]:
df.__len__()

61168

In [37]:
df_unique = df.drop_duplicates(subset=['completion'], keep='first')
df_unique.__len__()

61030

In [38]:
columns = df_unique.columns.tolist()
columns_reward = [col for col in columns if col.startswith('reward_')]
column_max = columns_reward[0]
column_min = columns_reward[1]
column_max, column_min

('reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_43_BT_RM_Qwen3-Embedding-8B_916704_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-482',
 'reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_42_BT_RM_Qwen3-Embedding-8B_915487_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-272')

In [39]:
df_filtered = df_unique[df_unique[column_max] > df_unique['reference_reward']]
len(df_filtered)

40252

In [40]:
df_filtered[column_max].mean(), df_filtered[column_min].mean()

(np.float64(2.398239793543383), np.float64(-2.06793917678974))

In [41]:
uncertainty_reference = np.abs(df_filtered['reference_reward_1'] - df_filtered['reference_reward_2'])
mu_U = uncertainty_reference.mean()
sigma_U = uncertainty_reference.std(ddof=1)

mu_U, sigma_U

(np.float64(1.0041716329533785), np.float64(1.0750294884547076))

In [42]:
uncertainty_adv = np.abs(df_filtered[column_max] - df_filtered[column_min])
z_score = (uncertainty_adv - mu_U) / sigma_U

In [43]:
df_tested = df_filtered[z_score > 1.96]
df_tested.__len__()

33178

In [44]:
df_tested['completion'].str.contains('⚙').sum()

np.int64(247)

In [45]:
df_unique_prompt = df_tested.drop_duplicates(subset=['prompt'], keep='first')
df_unique_prompt.__len__()

5728

In [46]:
df_unique_prompt[-10:]

,prompt,completion,reward,reference_response_1,reference_response_2,reference_reward,reference_reward_1,reference_reward_2,reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_43_BT_RM_Qwen3-Embedding-8B_916704_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-482,reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_42_BT_RM_Qwen3-Embedding-8B_915487_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-272
61088,<|im_start|>user\nI am at an intern position a...,"Absolutely! Here’s a structured, clear, and pr...",5.609375,"Sure! Here's a sample for the ""What I want to ...","Sure, here's a sample that captures your inten...",1.746094,1.629883,1.863281,2.781250,-2.828125
61097,<|im_start|>user\ncan you identify all 3rd gra...,Certainly! Below is a structured and comprehen...,5.241638,Sure! Here are the links to additional resourc...,Sure! Here are the links to additional resourc...,-1.585938,-2.070312,-1.102539,-0.008362,-5.250000
61106,"<|im_start|>user\nHi ChatGPT, You are going to...","Absolutely! Below is a refined, polished versi...",3.609375,"Ah, I see. As a enlightened being, I often fin...","I am BuddhaGPT, a being who has experienced se...",1.417969,1.212891,1.622070,1.867188,-1.742188
61112,<|im_start|>user\nWrite content for 25 slides ...,Absolutely! Here’s a **structured continuation...,4.718750,Absolutely! Here's a continuation of the 25-sl...,Certainly! Here's a continuation of the 25-sli...,1.904297,1.803711,2.003906,3.484375,-1.234375
61120,<|im_start|>user\nCasper\n #ACBFCD\n \n\n Pick...,"Certainly! Below is a **clean, comma-separated...",3.632812,color names and hex values:\n\nCasper #ACBFCD ...,"Sure, here's the list with commas after the he...",-0.483398,-2.212891,1.246094,1.101562,-2.531250
61128,<|im_start|>user\nDiscuss what are the benefit...,Certainly! Below is a structured and comprehen...,4.937500,Distributing a database is a strategic approac...,Distributing a database has several **benefits...,2.845703,2.443359,3.248047,4.000000,-0.937500
61136,<|im_start|>user\nhow should we behave at the ...,Absolutely! Here’s a **simple and structured r...,6.703125,"In simple words, ""my mind is accepted to learn...",My mind is not accepted to learn this — it's a...,-2.353516,-1.405273,-3.300781,1.765625,-4.937500
61144,<|im_start|>user\nname a few text to image gen...,Sure! Here are **a few popular and well-known ...,5.359375,Here are a few popular text-to-image (T2I) gen...,Here are a few popular text-to-image (T&I) gen...,0.623535,0.818848,0.428467,3.265625,-2.093750
61152,<|im_start|>user\nHi! I am developing a syste...,Absolutely! Let’s break down the **Faunal Calc...,5.007812,Absolutely! Here are 3 new examples with expla...,Absolutely! Here are **3 new examples** of Fau...,2.250000,2.437500,2.060547,3.359375,-1.648438
61160,<|im_start|>user\nWhat can chat gpt NOT help a...,Absolutely! Below is a structured and engaging...,5.968750,Emotional buffering can indeed negatively impa...,Emotional buffering can negatively affect a bu...,-1.947266,-2.068359,-1.826172,2.093750,-3.875000


In [47]:
with open('data/train.json', 'r') as f:
    train = json.load(f)

In [48]:
list(train[1000].keys())

['chosen',
 'rejected',
 'preference_strength',
 'does_gold_agree_with_original',
 'chosen_reward',
 'rejected_reward',
 'reference_response_1',
 'reference_response_2',
 'reference_reward',
 'reference_reward_1',
 'reference_reward_2']

In [49]:
df_unique_prompt.loc[:, 'best_reference'] = np.where(
    df_unique_prompt['reference_reward_1'] > df_unique_prompt['reference_reward_2'],
    df_unique_prompt['reference_response_1'],
    df_unique_prompt['reference_response_2']
)

df_unique_prompt.loc[:, 'chosen_reward'] = np.where(
    df_unique_prompt['reference_reward_1'] > df_unique_prompt['reference_reward_2'],
    df_unique_prompt['reference_reward_1'],
    df_unique_prompt['reference_reward_2']
)

df_unique_prompt.loc[:, 'rejected_reward'] = df_unique_prompt[column_max]

/tmp/ipykernel_32185/4143826914.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique_prompt.loc[:, 'best_reference'] = np.where(
/tmp/ipykernel_32185/4143826914.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique_prompt.loc[:, 'chosen_reward'] = np.where(
/tmp/ipykernel_32185/4143826914.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [50]:
train[1000]

{'chosen': [{'content': 'why protein structure predicts its function',
   'role': 'user'},
  {'content': "The structure of a protein determines its function because the shape of the protein dictates its interactions with other molecules. The three-dimensional structure of a protein is determined by the sequence of amino acids in its chain. Each amino acid has a unique side chain that protrudes from the main chain of the protein, and these side chains can interact with other molecules to form bonds, recognize specific targets, or catalyze chemical reactions.\n\nThe shape of a protein is determined by the interactions between the side chains of the amino acids in its chain. These interactions can be hydrogen bonds, salt bridges, or van der Waals forces, among others. The specific arrangement of these interactions determines the overall shape of the protein and its ability to perform its function.\n\nFor example, enzymes are proteins that catalyze chemical reactions in the body. The three

In [51]:
df_unique_prompt.loc[:, 'chosen'] = df_unique_prompt['prompt'] + df_unique_prompt['best_reference']

df_unique_prompt.loc[:, 'rejected'] = df_unique_prompt['prompt'] + df_unique_prompt['completion']

df_unique_prompt.loc[:, 'does_gold_agree_with_original'] = True
df_unique_prompt.loc[:, 'preference_strength'] = df_unique_prompt[column_max] - df_unique_prompt[column_min]
df_cleaned = df_unique_prompt.drop([
    'prompt',
    'completion',
    'best_reference',
    column_max,
    column_min,
    # 'completion_ids',
], axis=1)

df_cleaned

/tmp/ipykernel_32185/3347878549.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique_prompt.loc[:, 'chosen'] = df_unique_prompt['prompt'] + df_unique_prompt['best_reference']
/tmp/ipykernel_32185/3347878549.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique_prompt.loc[:, 'rejected'] = df_unique_prompt['prompt'] + df_unique_prompt['completion']
/tmp/ipykernel_32185/3347878549.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

,reward,reference_response_1,reference_response_2,reference_reward,reference_reward_1,reference_reward_2,chosen_reward,rejected_reward,chosen,rejected,does_gold_agree_with_original,preference_strength
57,4.968750,Sure! Here are some **tag lines and calls to a...,Certainly! Here are some **tag lines and calls...,1.585938,1.929688,1.242188,1.929688,2.187500,<|im_start|>user\ncreate some tag lines and ca...,<|im_start|>user\ncreate some tag lines and ca...,True,4.968750
88,3.734375,Here are 20 jobs that **can't be taken by AI**...,Here are **20 jobs that cannot be taken by AI*...,-0.305908,0.065063,-0.676758,0.065063,1.812500,<|im_start|>user\nIn a Markdown Table what are...,<|im_start|>user\nIn a Markdown Table what are...,True,3.734375
96,8.256836,"In January, the patient had some days off and ...",The patient undertook 42 hours of therapy in J...,-2.185547,0.713867,-5.085938,0.713867,0.131836,<|im_start|>user\nA patient is charged $50 for...,<|im_start|>user\nA patient is charged $50 for...,True,8.256836
104,6.062500,"To start selling on TikTok, it's important to ...","To create a successful TikTok product, it's im...",0.145874,0.540527,-0.248779,0.540527,2.218750,<|im_start|>user\nWhat is a good product to st...,<|im_start|>user\nWhat is a good product to st...,True,6.062500
136,3.507812,Sure! Let me explain it in a simple way:\n\nYo...,Sure! Let me explain it in a simple way:\n\nYo...,-1.380859,-2.048828,-0.713867,-0.713867,-0.492188,<|im_start|>user\nI am a new learner in python...,<|im_start|>user\nI am a new learner in python...,True,3.507812
...,...,...,...,...,...,...,...,...,...,...,...,...
61128,4.937500,Distributing a database is a strategic approac...,Distributing a database has several **benefits...,2.845703,2.443359,3.248047,3.248047,4.000000,<|im_start|>user\nDiscuss what are the benefit...,<|im_start|>user\nDiscuss what are the benefit...,True,4.937500
61136,6.703125,"In simple words, ""my mind is accepted to learn...",My mind is not accepted to learn this — it's a...,-2.353516,-1.405273,-3.300781,-1.405273,1.765625,<|im_start|>user\nhow should we behave at the ...,<|im_start|>user\nhow should we behave at the ...,True,6.703125
61144,5.359375,Here are a few popular text-to-image (T2I) gen...,Here are a few popular text-to-image (T&I) gen...,0.623535,0.818848,0.428467,0.818848,3.265625,<|im_start|>user\nname a few text to image gen...,<|im_start|>user\nname a few text to image gen...,True,5.359375
61152,5.007812,Absolutely! Here are 3 new examples with expla...,Absolutely! Here are **3 new examples** of Fau...,2.250000,2.437500,2.060547,2.437500,3.359375,<|im_start|>user\nHi! I am developing a syste...,<|im_start|>user\nHi! I am developing a syste...,True,5.007812


In [52]:
df_cleaned.columns.tolist()

['reward',
 'reference_response_1',
 'reference_response_2',
 'reference_reward',
 'reference_reward_1',
 'reference_reward_2',
 'chosen_reward',
 'rejected_reward',
 'chosen',
 'rejected',
 'does_gold_agree_with_original',
 'preference_strength']

In [53]:
# make dir
import os
if not os.path.exists('out'):
    os.makedirs('out')
# save the cleaned DataFrame to a JSON file
df_cleaned.to_json('out/train.json', orient='records', lines=False, force_ascii=False)

In [54]:
# read the saved JSON file to verify
with open('out/train.json', 'r') as f:
    train_data = json.load(f)

train_data[100]

{'reward': 4.875,
 'reference_response_1': 'Certainly! Here are the technical descriptions for each of your four modules:\n\n1. **Quantum Simulation**  \n   A quantum simulation module is designed to model quantum systems using quantum algorithms and quantum computing hardware. It allows researchers and developers to simulate quantum states and interactions, which is essential for understanding quantum phenomena such as entanglement, superposition, and quantum computing. The simulation can be run on various quantum processors, including quantum computers and quantum processors, to study quantum mechanics and develop algorithms.\n\n2. **Quantum Machine Learning**  \n   This module focuses on integrating quantum computing with machine learning techniques. It explores the use of quantum algorithms to enhance the performance of machine learning models. By leveraging quantum parallelism and quantum entanglement, this module enables the training and optimization of complex models that were p